# 10. Traits
(Jason Swartz - Learning Scala_ Practical Functional Programming for the JVM (2014, O'Reilly Media)

A trait is a kind of class that enables multiple inheritance. Classes, case classes, objects,
and (yes) traits can all extend no more than one class but can extend multiple traits at
the same time. Unlike the other types, however, traits cannot be instantiated.
Traits look about the same as any other type of class. However, like objects, they cannot
take class parameters. Unlike objects, however, traits can take type parameters, which
can help to make them extremely reusable.
To define a trait, use the trait keyword in place of where you would normally use the
class keyword.

Syntax: Defining a Trait

`trait <identifier> [extends <identifier>] [{ fields, methods, and classes }]`

Let's implement a trait:

In [1]:
trait HtmlUtils {
  def removeMarkup(input: String) = {
  input
  .replaceAll("""</?\w[^>]*>""","")
  .replaceAll("<.*>","")
  }
}

defined trait HtmlUtils

In [2]:
class Page(val s: String) extends HtmlUtils {
  def asPlainText = removeMarkup(s)
  }


defined class Page

In [4]:
val mypage = new Page("<html><body><h1>Introduction</h1></body></html>").asPlainText

mypage: String = "Introduction"

Our Page class can now use the removeMarkup method directly without specifying an
object name.

#### Note: Traits Come After the Parent Class
If you are extending a class and one or more traits, you will need to
extend the class before you can add the traits using the with key‐
word. A parent class, if specified, must always come before any par‐
ent traits.


This works pretty well, but a class version of HtmlUtils could have done the same job.
Let’s make it more interesting by adding a second trait. This time we’ll use a new key‐
word, with, which is required for extending the second and later traits:


In [5]:
 trait SafeStringUtils {
  // Returns a trimmed version of the string wrapped in an Option,
  // or None if the trimmed string is empty.
  def trimToNone(s: String): Option[String] = {
  Option(s) map(_.trim) filterNot(_.isEmpty)
  }
  }


defined trait SafeStringUtils

In [6]:
 class Page(val s: String) extends SafeStringUtils with HtmlUtils {
  def asPlainText: String = {
  trimToNone(s) map removeMarkup getOrElse "n/a"
  }
 }


defined class Page

In [10]:
new Page("<html><body><h1>Introduction</h1></body></html>").asPlainText
new Page(" ").asPlainText
 new Page(null).asPlainText

res9_0: String = "Introduction"
res9_1: String = "n/a"
res9_2: String = "n/a"

Our new, more robust Page class now extends two traits and can handle null or empty
strings by returning the message n/a.
If you’re familiar with the JVM you may be wondering how Scala can support multiple
inheritance with traits. After all, JVM classes can only extend one parent class. The
answer is that although the language supports multiple inheritance in theory, the com‐
piler actually creates copies of each trait to form a tall, single-column hierarchy of the
class and traits. So, a class extending class A and traits B and C is actually extending one
class, which extends another class, which extends another class, when compiled to
the .class binary file.
This process of taking a horizontal list of a class and traits being extended, and reforming
them into a vertical chain of one class extending another, is known as linearization. It
is a kind of coping mechanism for supporting multiple inheritance in an execution
environment that only supports single inheritance. The fact that the JVM only supports
single inheritance ensures that all class hierarchies are nondeterministic and prevents
the possibility of confusing two traits that have competing members.


#### What Happens If You Have Traits with Competing Members?
What Happens If You Have Traits with Competing Members?
A class importing two traits that have the same field or method, but
lack an override keyword, will fail to compile. The compilation er‐
ror is the same as if you were extending a class and providing your
own version of a method but failed to add an override keyword. In
the case of the traits, adding a common base class and then overrid‐
ing the field or method with the override keyword will ensure the
traits can be extended by the same class.